In [110]:
rm(list = ls())

library(dplyr)
library(ggplot2)
library(readr)
library(tidyverse)
library(viridis)
library(ggmap)
library(sf)
library(data.table)
library(bit64)
library(corrplot)

setwd("C:/Users/momaleki/Desktop/Project")
getwd()

df_No_SCI <- fread("Data/ToMerge/df_No_SCI.csv")
df_No_SCI <- df_No_SCI[, -c(1)]
df <- df_No_SCI
df <- df[, -c(8:9, 11:14, 26:28 )]


colnames(df) <- c( 'NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS', 
                   'FIPS', 'Combined','State', 'Percent_Less_HS', 'Percent_HS', 'Percent_Less_College', 'Percent_Higher_College', 
                 'Trump_Win', 'Biden_Win', 'LAT', 'Lon', 'Unemployment', 'Movement_Change', 'Percent_InTouch', 
                 'Percent_Rural', 'Pop_Density', 'Poverty', 'COVID_Confirmed', 'COVID_Death', 'Percent_Ess_Workers', 
                 'Hospital_AllBeds', 'Hospital_Occ_rate', 'COVID_TestCap', 'COVID_Testing', 'COVID_Clinic', 
                 'Vaccine', 'Crime')


df <- df[, -c(14:15)]
head(df[, c(1:15)])

[1] "C:/Users/momaleki/Desktop/Project"

NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,FIPS,Combined,State,Percent_Less_HS,Percent_HS,Percent_Less_College,Percent_Higher_College,Trump_Win,Lon,Unemployment
0.026,0.006,0.106,0.051,0.811,10001,4.615,Delaware,0.126,0.329,0.308,0.237,0,-75.50298,4.2
0.026,0.006,0.106,0.051,0.811,10001,4.615,Delaware,0.126,0.329,0.308,0.237,0,-75.50298,4.2
0.026,0.006,0.106,0.051,0.811,10001,4.615,Delaware,0.126,0.329,0.308,0.237,0,-75.50298,4.2
0.026,0.006,0.106,0.051,0.811,10001,4.615,Delaware,0.126,0.329,0.308,0.237,0,-75.50298,4.2
0.027,0.013,0.041,0.133,0.786,10003,4.638,Delaware,0.084,0.308,0.246,0.362,0,-75.64413,3.7
0.027,0.013,0.041,0.133,0.786,10003,4.638,Delaware,0.084,0.308,0.246,0.362,0,-75.64413,3.7


In [111]:
dataFrame <- data.frame(matrix( ncol = 0 , nrow = 4 ))
states <- sort(unique(df$State))

In [112]:
i <- 0
for (state in states){
    i <- i + 1
    print(paste0((i), "---->", state))
}

[1] "1---->Delaware"
[1] "2---->Florida"
[1] "3---->Georgia"
[1] "4---->Hawaii"
[1] "5---->Idaho"
[1] "6---->Illinois"
[1] "7---->Indiana"
[1] "8---->Iowa"
[1] "9---->Kansas"
[1] "10---->Kentucky"
[1] "11---->Louisiana"
[1] "12---->Maine"
[1] "13---->Maryland"
[1] "14---->Michigan"
[1] "15---->Minnesota"
[1] "16---->Mississippi"
[1] "17---->Missouri"
[1] "18---->Montana"
[1] "19---->Nebraska"
[1] "20---->Nevada"
[1] "21---->New Jersey"
[1] "22---->New Mexico"
[1] "23---->New York"
[1] "24---->North Carolina"
[1] "25---->North Dakota"
[1] "26---->Ohio"
[1] "27---->Oklahoma"
[1] "28---->Oregon"
[1] "29---->Pennsylvania"
[1] "30---->South Carolina"
[1] "31---->South Dakota"
[1] "32---->Tennessee"
[1] "33---->Texas"
[1] "34---->Utah"
[1] "35---->Virginia"
[1] "36---->Washington"
[1] "37---->West Virginia"
[1] "38---->Wisconsin"
[1] "39---->Wyoming"


In [123]:
dataFrame <- data.frame(matrix( ncol = 0 , nrow = 4 ))
states <- sort(unique(df$State))

i <- 0

for (state in states){

    
    mydata = df[df$State == state]
    
    myreg_mask <-  lm(data = mydata, 
                  df$ALWAYS ~  log1p(df$Pop_Density) + 
                   df$Percent_Higher_College + 
                   log1p(df$Poverty) + 
                   log1p(df$Unemployment) + 
                  log1p(df$COVID_Testing) + 
                   df$COVID_Clinic + 
                 df$Percent_Ess_Workers + 
                  log1p(df$COVID_Confirmed) +
                 df$Hospital_Occ_rate + 
                  df$Trump_Win +
                log1p(df$Crime))
    
    myreg_move <-  lm(data = mydata, 
                  df$Movement_Change ~  log1p(df$Pop_Density) + 
                   df$Percent_Higher_College + 
                   log1p(df$Poverty) + 
                   log1p(df$Unemployment) + 
                  log1p(df$COVID_Testing) + 
                   df$COVID_Clinic + 
                 df$Percent_Ess_Workers + 
                  log1p(df$COVID_Confirmed) +
                 df$Hospital_Occ_rate + 
                  df$Trump_Win +
                 log1p(df$Crime))


    myreg_vaccine <-  lm(data = mydata, 
                  df$Vaccine ~  log1p(df$Pop_Density) + 
                   df$Percent_Higher_College + 
                   log1p(df$Poverty) + 
                   log1p(df$Unemployment) + 
                  log1p(df$COVID_Testing) + 
                   df$COVID_Clinic + 
                 df$Percent_Ess_Workers + 
                  log1p(df$COVID_Confirmed) +
                 df$Hospital_Occ_rate + 
                  df$Trump_Win +
                 log1p(df$Crime))


    
    t_val_ranked_Mask <- row.names(data.frame(head(summary(myreg_mask)$coefficients[,3][order(-abs(summary(myreg_mask)$coefficients[,3]))],
                                               4)))
    dataFrame[, paste0(state, "_Mask",  sep = "")] = t_val_ranked_Mask




    t_val_ranked_Move <- row.names(data.frame(head(summary(myreg_move)$coefficients[,3][order(-abs(summary(myreg_move)$coefficients[,3]))],
                                               4)))
    dataFrame[, paste0(state, "_Movement", sep = "")] = t_val_ranked_Move


    t_val_ranked_Vaccine <- row.names(data.frame(head(summary(myreg_vaccine)$coefficients[,3][order(-abs(summary(myreg_vaccine)$coefficients[,3]))],
                                               4)))
    dataFrame[, paste0(state, "_Vaccine", sep = "")] = t_val_ranked_Vaccine
    
    i <- i + 1
    
    print(t_val_ranked_Mask)
}

dataFrame


[1] "(Intercept)"             "df$Trump_Win"           
[3] "log1p(df$COVID_Testing)" "log1p(df$Crime)"        
[1] "(Intercept)"             "df$Trump_Win"           
[3] "log1p(df$COVID_Testing)" "log1p(df$Crime)"        
[1] "(Intercept)"             "df$Trump_Win"           
[3] "log1p(df$COVID_Testing)" "log1p(df$Crime)"        
[1] "(Intercept)"             "df$Trump_Win"           
[3] "log1p(df$COVID_Testing)" "log1p(df$Crime)"        
[1] "(Intercept)"             "df$Trump_Win"           
[3] "log1p(df$COVID_Testing)" "log1p(df$Crime)"        
[1] "(Intercept)"             "df$Trump_Win"           
[3] "log1p(df$COVID_Testing)" "log1p(df$Crime)"        
[1] "(Intercept)"             "df$Trump_Win"           
[3] "log1p(df$COVID_Testing)" "log1p(df$Crime)"        
[1] "(Intercept)"             "df$Trump_Win"           
[3] "log1p(df$COVID_Testing)" "log1p(df$Crime)"        
[1] "(Intercept)"             "df$Trump_Win"           
[3] "log1p(df$COVID_Testing)" "log1p(df$Crime)" 

Delaware_Mask,Delaware_Movement,Delaware_Vaccine,Florida_Mask,Florida_Movement,Florida_Vaccine,Georgia_Mask,Georgia_Movement,Georgia_Vaccine,Hawaii_Mask,...,Washington_Vaccine,West Virginia_Mask,West Virginia_Movement,West Virginia_Vaccine,Wisconsin_Mask,Wisconsin_Movement,Wisconsin_Vaccine,Wyoming_Mask,Wyoming_Movement,Wyoming_Vaccine
(Intercept),(Intercept),log1p(df$COVID_Confirmed),(Intercept),(Intercept),log1p(df$COVID_Confirmed),(Intercept),(Intercept),log1p(df$COVID_Confirmed),(Intercept),...,log1p(df$COVID_Confirmed),(Intercept),(Intercept),log1p(df$COVID_Confirmed),(Intercept),(Intercept),log1p(df$COVID_Confirmed),(Intercept),(Intercept),log1p(df$COVID_Confirmed)
df$Trump_Win,log1p(df$Pop_Density),df$Hospital_Occ_rate,df$Trump_Win,log1p(df$Pop_Density),df$Hospital_Occ_rate,df$Trump_Win,log1p(df$Pop_Density),df$Hospital_Occ_rate,df$Trump_Win,...,df$Hospital_Occ_rate,df$Trump_Win,log1p(df$Pop_Density),df$Hospital_Occ_rate,df$Trump_Win,log1p(df$Pop_Density),df$Hospital_Occ_rate,df$Trump_Win,log1p(df$Pop_Density),df$Hospital_Occ_rate
log1p(df$COVID_Testing),df$Percent_Higher_College,log1p(df$Crime),log1p(df$COVID_Testing),df$Percent_Higher_College,log1p(df$Crime),log1p(df$COVID_Testing),df$Percent_Higher_College,log1p(df$Crime),log1p(df$COVID_Testing),...,log1p(df$Crime),log1p(df$COVID_Testing),df$Percent_Higher_College,log1p(df$Crime),log1p(df$COVID_Testing),df$Percent_Higher_College,log1p(df$Crime),log1p(df$COVID_Testing),df$Percent_Higher_College,log1p(df$Crime)
log1p(df$Crime),log1p(df$COVID_Confirmed),log1p(df$COVID_Testing),log1p(df$Crime),log1p(df$COVID_Confirmed),log1p(df$COVID_Testing),log1p(df$Crime),log1p(df$COVID_Confirmed),log1p(df$COVID_Testing),log1p(df$Crime),...,log1p(df$COVID_Testing),log1p(df$Crime),log1p(df$COVID_Confirmed),log1p(df$COVID_Testing),log1p(df$Crime),log1p(df$COVID_Confirmed),log1p(df$COVID_Testing),log1p(df$Crime),log1p(df$COVID_Confirmed),log1p(df$COVID_Testing)


In [124]:
mydata = df[df$State == 'Wyoming']
    
myreg_mask <-  lm(data = mydata, 
                  df$ALWAYS ~  log1p(df$Pop_Density) + 
                   df$Percent_Higher_College + 
                   log1p(df$Poverty) + 
                   log1p(df$Unemployment) + 
                  log1p(df$COVID_Testing) + 
                   df$COVID_Clinic + 
                 df$Percent_Ess_Workers + 
                  log1p(df$COVID_Confirmed) +
                 df$Hospital_Occ_rate + 
                  df$Trump_Win +
                log1p(df$Crime))

t_val_ranked_Mask <- row.names(data.frame(head(summary(myreg_mask)$coefficients[,3][order(-abs(summary(myreg_mask)$coefficients[,3]))],
                                               4)))

print(t_val_ranked_Mask)

[1] "(Intercept)"             "df$Trump_Win"           
[3] "log1p(df$COVID_Testing)" "log1p(df$Crime)"        


In [126]:
mydata = df[df$State == 'New York']
    
myreg_mask <-  lm(data = mydata, 
                  df$ALWAYS ~  log1p(df$Pop_Density) + 
                   df$Percent_Higher_College + 
                   log1p(df$Poverty) + 
                   log1p(df$Unemployment) + 
                  log1p(df$COVID_Testing) + 
                   df$COVID_Clinic + 
                 df$Percent_Ess_Workers + 
                  log1p(df$COVID_Confirmed) +
                 df$Hospital_Occ_rate + 
                  df$Trump_Win +
                log1p(df$Crime))

t_val_ranked_Mask <- row.names(data.frame(head(summary(myreg_mask)$coefficients[,3][order(-abs(summary(myreg_mask)$coefficients[,3]))],
                                               4)))

print(t_val_ranked_Mask)

[1] "(Intercept)"             "df$Trump_Win"           
[3] "log1p(df$COVID_Testing)" "log1p(df$Crime)"        
